In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

tokenizer = AutoTokenizer.from_pretrained("yhavinga/t5-v1.1-base-dutch-cnn-test")

model = AutoModelForSeq2SeqLM.from_pretrained("yhavinga/t5-v1.1-base-dutch-cnn-test")

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from datasets import load_dataset, load_metric

metric = load_metric("rouge")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
df_canon = pd.read_csv('df_canon_par.csv', sep = ';')

In [ ]:
## Split in train and test
train_all, test = train_test_split(df_canon, test_size=0.15)

In [ ]:
train, val = train_test_split(train_all, test_size=0.15)

In [ ]:
from datasets import Dataset
finetune_train = Dataset.from_pandas(train)
finetune_eval = Dataset.from_pandas(val)

In [ ]:
if model in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
## Function for tokenizing data
# Random paramaters, should be adjusted and tested
max_input_length = 5000
max_target_length = 4000

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, padding='max_length', truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, padding='max_length', truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
tokenized_datasets_train = finetune_train.map(preprocess_function, batched=True)

In [ ]:
tokenized_datasets_eval = finetune_eval.map(preprocess_function, batched=True)

In [ ]:
batch_size = 2
model_name =  "Dutch_canon"
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    push_to_hub=True
)


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_eval
)


In [ ]:
trainer.train()